### 1. Write a format string (using `str.format()` and `f'string'`) that will take the following four element tuple:

`( 2, 123.4567, 10000, 12345.67)`

and produce:
    
`'file_002 :   123.46, 1.00e+04, 1.23e+04'`

### 2. Write a python function that output the following formatted string based on the input iterable:

`'The n numbers are: i1, i2, i3, ...'`
    
where `n` is the number in the iterable and `i1, i2 ...` are the items inside the iterable. 

for example:

`func([1, 2, 3])` will return `'The 3 numbers are: 1, 2, 3'`

`func((6, 7, 8, 9))` will return `'The 4 numbers are: 6, 7, 8, 9'`

### 3. Write a similar function as the last one but it inputs variable-length arguments instead

`func(1, 2, 3)` will return `'The 3 numbers are: 1, 2, 3'`

`func(6, 7, 8, 9)` will return `'The 4 numbers are: 6, 7, 8, 9'`

### 4. Write a format string (f'string') that will take the following tuple and return the formatted string below:

Tuple: `(4, 30, 2017, 2, 27)`

Return: `'02 27 2017 04 30'`

### 5. Write a format string (f'string') that will take the following input and return the formatted string below:

Input: `['oranges', 1.3, 'lemons', 1.1]`
    
Return: `'The weight of an orange is 1.3 and the weight of a lemon is 1.1 and the total weight is 2.4'`

### 6. Write a function that will format each item in an input iterable object into a customized width 

syntax: `func(iterable, width)`

for example:
    
`func([1, 2, 3, 4, 5], 3)`  return `'  1  2  3  4  5'`

`func([1, 2, 3, 4, 5], 2)`  return `' 1 2 3 4 5'`

### 7. The USGS Water Data service provides retrival of streamflow data at the USGS stream gages. The download url follows the pattern below. Write a Python program 1) generate the url for a given list of gage numbers; 2) read this url and download the data to csv file (do not use numpy or pandas); 3) based on the downloaded data calculate the monthly stasticstics including maximum, minimum and average of the streamflow data and save it to another csv file. The data need to be formatted to 2 decimal digits. 4) calculate the average annual runoff of each gage (expressed in acre-feet) 


url: https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb&site_no=06803495&referred_module=sw&period=&begin_date=2019-02-11&end_date=2020-02-11

where `&site_no=06803495` define which gages are included in this data retrieval; for multiple gages, simply repeat it, e.g. `&site_no=06803495&site_no=06803486`;

the datas are specified by `&period=&begin_date=2018-02-11&end_date=2020-02-11`

You could test your program using the gages `['06803495', '06803486']` for the period between `2000-10-1` and `2019-9-30`

Hint 1: using the str.format() method to construct the url.

Hint 2: use the `urllib.request.urlopen()` to download the url:
```python
    import urllib.request.urlopen
    f = urllib.request.urlopen(a_url)
    lines = f.readlines()
```

Hint 3: skip the lines starts with `#` in the downloaded data.

Hint 4: print some of the downloaded lines and identify the elements in each line.

Hint 5: using the str.format() or f-string method to write `csv` file.

Hint 6: the monthly stasticstics refer to the time series of monthly stasticstics.

Hint 7: for the average annual runoff, you need to calculate the total runoff for each year and then calculate average of these annual runoff.

In [15]:
from urllib.request import urlopen
import math
import datetime

def generate_url(gagelist, begin_date, end_date):
    try:
        bdate = datetime.date.fromisoformat(begin_date)
        edate = datetime.date.fromisoformat(end_date)
    except:
        raise ValueError ("The input formats for the begin_date and end_date must be YYYY-MM-DD")
        
    if type(gagelist) is not list:
        raise TypeError('The gagelist must be a list type.')
    
    if gagelist == []:
        raise ValueError('The gagelist must not be an empty list.')
        
    gages = ('&site_no={}' * len(gagelist)).format(*gagelist)
    period = f'&period=&begin_date={begin_date}&end_date={end_date}'
    return f'https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb{gages}&referred_module=sw{period}'

print(generate_url(['06803495', '06803486'], '2000-10-10', '2019-09-30'))

https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb&site_no=06803495&site_no=06803486&referred_module=sw&period=&begin_date=2000-10-10&end_date=2020-02-11


In [17]:
def read_usgs_flowdata(url):
    f = urllib.request.urlopen(url)
    return f.readlines()

lines = read_usgs_flowdata(generate_url(['06803495', '06803486'], '2000-10-10', '2019-09-30'))


In [20]:
for l in lines[:40]:
    print(l)

b'# ---------------------------------- WARNING ----------------------------------------\n'
b'# Some of the data that you have obtained from this U.S. Geological Survey database\n'
b"# may not have received Director's approval. Any such data values are qualified\n"
b'# as provisional and are subject to revision. Provisional data are released on the\n'
b'# condition that neither the USGS nor the United States Government may be held liable\n'
b'# for any damages resulting from its use.\n'
b'#\n'
b'# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement\n'
b'#\n'
b'# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output\n'
b'# Automated-retrieval info: https://help.waterdata.usgs.gov/faq/automated-retrievals\n'
b'#\n'
b'# Contact:   gs-w_support_nwisweb@usgs.gov\n'
b'# retrieved: 2020-02-12 13:04:24 EST       (sdww01)\n'
b'#\n'
b'# Data for the following 2 site(s) are contained in this file\n'
b'#    USGS 06803486 Oak Creek 

In [29]:
lines[39].decode().rstrip().split('\t')

['USGS', '06803486', '2000-10-18', '11.0', 'A']

In [34]:
def save_date(data_lines, csvfile):
    for n, l in enumerate(data_lines):
        if not l.startswith(b'#'):
            break
    
    # now start working on the lines containing data
    with open(csvfile, 'w') as f:
        # extract and write headers
        f.write(data_lines[n].decode().replace('\t', ','))
        
        # skip the column spec line
        n += 2
        for i, l in enumerate(data_lines[n:]):
            if l.startswith(b'USGS'):
                f.write(l.decode().replace('\t', ','))
                
save_date(lines, 'usgs_streamflow.csv') 

! head -20 'usgs_streamflow.csv'

agency_cd,site_no,datetime,93535_00060_00003,93535_00060_00003_cd
USGS,06803486,2000-10-10,10.0,A
USGS,06803486,2000-10-11,10.0,A
USGS,06803486,2000-10-12,11.0,A
USGS,06803486,2000-10-13,12.0,A
USGS,06803486,2000-10-14,13.0,A
USGS,06803486,2000-10-15,11.0,A
USGS,06803486,2000-10-16,11.0,A
USGS,06803486,2000-10-17,11.0,A
USGS,06803486,2000-10-18,11.0,A
USGS,06803486,2000-10-19,11.0,A
USGS,06803486,2000-10-20,10.0,A
USGS,06803486,2000-10-21,9.60,A
USGS,06803486,2000-10-22,12.0,A
USGS,06803486,2000-10-23,13.0,A
USGS,06803486,2000-10-24,14.0,A
USGS,06803486,2000-10-25,16.0,A
USGS,06803486,2000-10-26,15.0,A
USGS,06803486,2000-10-27,14.0,A
USGS,06803486,2000-10-28,16.0,A


In [44]:
# write the file as binary
def save_date(data_lines, csvfile):
    for n, l in enumerate(data_lines):
        if not l.startswith(b'#'):
            break
    
    # now start working on the lines containing data
    with open(csvfile, 'wb') as f:
        # extract and write headers
        f.write(data_lines[n].replace(b'\t', b','))
        
        # skip the column spec line
        n += 2
        for i, l in enumerate(data_lines[n:]):
            if l.startswith(b'USGS'):
                f.write(l.replace(b'\t', b','))
                
save_date(lines, 'usgs_streamflow.csv') 

! head -20 'usgs_streamflow.csv'
! tail -20 'usgs_streamflow.csv'

agency_cd,site_no,datetime,93535_00060_00003,93535_00060_00003_cd
USGS,06803486,2000-10-10,10.0,A
USGS,06803486,2000-10-11,10.0,A
USGS,06803486,2000-10-12,11.0,A
USGS,06803486,2000-10-13,12.0,A
USGS,06803486,2000-10-14,13.0,A
USGS,06803486,2000-10-15,11.0,A
USGS,06803486,2000-10-16,11.0,A
USGS,06803486,2000-10-17,11.0,A
USGS,06803486,2000-10-18,11.0,A
USGS,06803486,2000-10-19,11.0,A
USGS,06803486,2000-10-20,10.0,A
USGS,06803486,2000-10-21,9.60,A
USGS,06803486,2000-10-22,12.0,A
USGS,06803486,2000-10-23,13.0,A
USGS,06803486,2000-10-24,14.0,A
USGS,06803486,2000-10-25,16.0,A
USGS,06803486,2000-10-26,15.0,A
USGS,06803486,2000-10-27,14.0,A
USGS,06803486,2000-10-28,16.0,A
USGS,06803495,2020-01-23,393,P
USGS,06803495,2020-01-24,392,P
USGS,06803495,2020-01-25,327,P
USGS,06803495,2020-01-26,311,P
USGS,06803495,2020-01-27,311,P
USGS,06803495,2020-01-28,297,P
USGS,06803495,2020-01-29,270,P
USGS,06803495,2020-01-30,264,P
USGS,06803495,2020-01-31,318,P
USGS,06803495,2020-02-01,410,P
USGS,06803495,20

In [69]:
import math
def mean(lst): 
    try:
        while math.nan in lst:
            lst.remove(math.nan)
        return sum(lst) / len(lst) if len(lst) > 0 else 0
    except:
        print(lst)
            
def cal_month_stat(daily_csv, monthly_csv):
    site = ''
    date = ''
    begin = True
    with open(daily_csv) as f:
        # skip the header
        f.readline()
        idx_site = 1
        idx_date = 2
        idx_val  = 3
        for i, l in enumerate(f):
            items = l.split(',')
            
            # if site or month change, update the key 
            if items[idx_site] != site or items[idx_date][:7] != date:
                site = items[idx_site]
                date = items[idx_date][:7]
                key = f'{site},{date}'
                if begin:
                    site_month = {key: []}
                    begin = False
                else:
                    site_month[key] = []
                    
            # append the data to the list        
            if items[idx_val].replace('.', '0').isnumeric():
                site_month[key].append(float(items[idx_val]))
            else:
                # some time it's frozen (ice)
                print(items)
    

    # write the data
    with open(monthly_csv, 'w') as fw:
        # write the header first
        fw.write('site_no,month,mean,min,max\n')
        for k in site_month:
            if site_month[k] == []:
                print(k, site_month[k]) 
            else:
                fw.write(f'{k}-01,{mean(site_month[k]):.2f},{min(site_month[k]):.2f},{max(site_month[k]):.2f}\n')
            
cal_month_stat('usgs_streamflow.csv', 'monthly_streamflow.csv')
! head -20 'monthly_streamflow.csv'
! tail -20 'monthly_streamflow.csv'

['USGS', '06803486', '2018-11-22', 'Ice', 'P\n']
['USGS', '06803486', '2018-11-23', 'Ice', 'P\n']
['USGS', '06803486', '2018-11-24', 'Ice', 'P\n']
['USGS', '06803486', '2018-11-25', 'Ice', 'P\n']
['USGS', '06803486', '2018-11-30', 'Ice', 'P\n']
['USGS', '06803486', '2018-12-01', 'Ice', 'P\n']
['USGS', '06803486', '2018-12-02', 'Ice', 'P\n']
['USGS', '06803486', '2018-12-03', 'Ice', 'P\n']
['USGS', '06803486', '2018-12-04', 'Ice', 'P\n']
['USGS', '06803486', '2018-12-27', 'Ice', 'P\n']
['USGS', '06803486', '2018-12-28', 'Ice', 'P\n']
['USGS', '06803486', '2019-01-06', 'Ice', 'P\n']
['USGS', '06803486', '2019-01-07', 'Ice', 'P\n']
['USGS', '06803486', '2019-01-08', 'Ice', 'P\n']
['USGS', '06803486', '2019-01-09', 'Ice', 'P\n']
['USGS', '06803495', '2018-12-01', 'Ice', 'P\n']
['USGS', '06803495', '2018-12-27', 'Ice', 'P\n']
['USGS', '06803495', '2019-01-05', 'Ice', 'P\n']
['USGS', '06803495', '2019-01-06', 'Ice', 'P\n']
['USGS', '06803495', '2019-01-07', 'Ice', 'P\n']
['USGS', '06803495',

In [71]:
            
def cal_annual_runoff(daily_csv, annual_runoff_csv):
    site = ''
    year = ''
    begin = True
    with open(daily_csv) as f:
        # skip the header
        f.readline()
        idx_site = 1
        idx_date = 2
        idx_val  = 3
        for i, l in enumerate(f):
            items = l.split(',')
            
            # if site or month change, update the key 
            if items[idx_site] != site or items[idx_date][:4] != year:
                site = items[idx_site]
                year = items[idx_date][:4]
                key = f'{site},{year}'
                if begin:
                    site_year = {key: []}
                    begin = False
                else:
                    site_year[key] = []
                    
            # append the data to the list        
            if items[idx_val].replace('.', '0').isnumeric():
                site_year[key].append(float(items[idx_val]) * 86400 / 43560)
    

    # write the data
    with open(annual_runoff_csv, 'w') as fw:
        # write the header first
        fw.write('site_no,year,runoff\n')
        for k in site_year:
            fw.write(f'{k}-01-01,{sum(site_year[k]):.2f}\n')
            
cal_annual_runoff('usgs_streamflow.csv', 'annual_runoff.csv')
! head -20 'annual_runoff.csv'
! tail -20 'annual_runoff.csv'

site_no,year,runoff
06803486,2000-01-01,2099.70
06803486,2001-01-01,34437.36
06803486,2002-01-01,13491.21
06803486,2003-01-01,10859.17
06803486,2004-01-01,8893.77
06803486,2005-01-01,11422.25
06803486,2006-01-01,12576.22
06803486,2007-01-01,62805.82
06803486,2008-01-01,95886.55
06803486,2009-01-01,31100.03
06803486,2010-01-01,71793.32
06803486,2011-01-01,44719.34
06803486,2012-01-01,16833.02
06803486,2013-01-01,16039.26
06803486,2014-01-01,70420.03
06803486,2015-01-01,68874.45
06803486,2016-01-01,79819.44
06803486,2017-01-01,71579.50
06803486,2018-01-01,47159.60
06803486,2017-01-01,71579.50
06803486,2018-01-01,47159.60
06803486,2019-01-01,141541.49
06803486,2020-01-01,10567.54
06803495,2005-01-01,46383.27
06803495,2006-01-01,44310.55
06803495,2007-01-01,169856.53
06803495,2008-01-01,252522.84
06803495,2009-01-01,74851.44
06803495,2010-01-01,214021.88
06803495,2011-01-01,143072.33
06803495,2012-01-01,66706.91
06803495,2013-01-01,78682.31
06803495,2014-01-01,147447.47
06803495,2015-01-01